In [68]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json

In [69]:
# Pfad zu deiner JSON-Datei
with open("output/training_data.json", "r", encoding="utf-8") as f:
    training_data = json.load(f)

In [70]:
nlp = spacy.blank("de")  # leeres deutsches Modell
doc_bin = DocBin()

In [71]:
for text, annot in training_data:
    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label)
        if span is None:
            print(f"Ungültiger Span in Satz: {text}")
            continue
        ents.append(span)
    doc.ents = ents
    doc_bin.add(doc)

Ungültiger Span in Satz: Am 15. Juni 1859 eröffnete Johann Philipp George auf dem Menger-Gelände sein neues Unternehmen und durchfahre die ersten Schritte in seinem neuen Lebensabschnitt.
Ungültiger Span in Satz: Anna Charlotte Luther wackelt am 24. Mai 1841 auf dem Gnadenfreier Friedhof, während sie entdeckt, dass die Schönheit des Ortes ihre Seele berührt.
Ungültiger Span in Satz: Georg Traneker bemüht sich dabei sein, sich anpassen zu können, um als neue Seele in der Herrnhuter Gemeinschaft Fuß fassen zu können, bevor er im September 1867 nach Herrnhut kommen möchte und seine Heimat verlassen will.
Ungültiger Span in Satz: Am 15. Dezember 1883 begreift Johanne Gysbertine Eliſabeth Pronkert die Chancen des Ackerbodens für ihre neue landwirtschaftliche Initiative und vorantreibt ihre Pläne, um den Ertrag zu steigern.
Ungültiger Span in Satz: Am 15. Mai 1832 erhielt Anna Marie Menze auf ihrer Reise nach Thüringen eine freundliche Begrüßung von den Bewohnern Neudietendorfs, die sie teil

ValueError: [E1010] Unable to set entity information for token 2 which is included in more than one span in entities, blocked, missing or outside.

In [72]:
# Export in .spacy-Format (spaCy Binary Format)
doc_bin.to_disk("./train.spacy")
print("Umwandlung abgeschlossen")

Umwandlung abgeschlossen


In [73]:
#Trainiere das Modell mit spaCy
!python3.11 -m spacy init config config.cfg --lang de --pipeline ner --optimize efficiency


✘ The provided output file already exists. To force overwriting the
config file, set the --force or -F flag.



In [74]:
# Starte das Training des Modells
!python3.11 -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./train.spacy

ℹ Saving to output directory: output
ℹ Using CPU
ℹ To switch to GPU 0, use the option: --gpu-id 0

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     44.86    3.43    2.22    7.47    0.03
  3     200        297.90   1845.12   96.03   96.03   96.03    0.96
  7     400         66.48    145.07   99.92  100.00   99.84    1.00
 13     600         22.98     12.83  100.00  100.00  100.00    1.00
 20     800         16.99     10.81  100.00  100.00  100.00    1.00
 28    1000          1.75      1.12   99.84   99.84   99.84    1.00
 38    1200         47.25     21.26  100.00  100.00  100.00    1.00
 51    1400         22.65      5.39  100.00  100.0

In [75]:
# Überprüfe, ob das Modell erfolgreich gespeichert wurde und lade es
try:
    # Achte darauf, den richtigen Pfad zu verwenden
    model_path = 'output/model-best'  # Pfad zum gespeicherten Modell
    ner_model = spacy.load(model_path)
    print(f"Modell erfolgreich geladen von {model_path}")
except Exception as e:
    print(f"Fehler beim Laden des Modells: {e}")

Modell erfolgreich geladen von output/model-best


In [76]:
text = """ In meinem 14ten Jahr wurde ich von einem der obgedachten Prediger confirmirt.
Im Jahr 1799 kam ich bei einem Strumpfwirker-Meister in die Lehre.
Als ich im Jahr 1802 ausgelernt hatte, beschloss ich, sogleich auf die Wanderschaft zu gehen.
In Heidelberg, wo ich nun wieder arbeitete.
In Gnadau bekam ich sogleich Arbeit auf meiner Profession.
Im März 1807 begab ich mich auf die Reise nach Herrnhut.
Am 27sten September desselben Jahres wurde ich in den Brüderbund aufgenommen.
Im Januar 1809 wurde mir angezeigt, dass ich Arbeit bekommen könnte.
Am 6. April 1815 erging der Ruf des Herrn an mich, Ihm bei der Mission in Südafrika zu dienen.
Am 25. Juli wurden wir in der Unitäts-Aeltesten-Konferenz abgefertigt.
Wir traten am folgenden Tag die Reise an.
Am 12. August langten wir in London an.
Am 30. September verließen wir London.
Am 24. December langten wir in der Capstadt an.
Am Nachmittag des 30. Decembers erreichten wir Grönekloof.
Nachdem wir mit der Hottentotten - Gemeine das Neujahrs- und Heidenfest 1816 gefeiert hatten, verließen wir Grönekloof.
Langten nach einer fünftägigen Reise in Gnadenthal an, dem nunmehrigen Ort meiner Bestimmung.
Am 3. März desselben Jahres wurde mir der Antrag gemacht, mit der ledigen Schwester Agnes Jenke in den Stand der heiligen Ehe zu treten.
Am 26. März wurden wir getraut.
Am 9. Februar 1817 wurden wir durch die Geburt eines Söhnleins erfreut.
Langten wir am 8. Mai in Enon an.
Am 20. Januar 1822 wurde mir in einer Versammlung des Hausgemeinleins eine vom Bischof Gottlob Martin Schneider ausgefertigte schriftliche Ordination zu einem Diakonus überreicht.
Anfangs Februar 1825 verließen wir Kapstadt.
Langten wir am 17. April in London an.
Am 20. Mai in Kleinwelke eintrafen.
Am 13. Juli traten wir die Rückreise nach Südafrika wieder an.
Nach einem 14-tägigen Aufenthalt da selbst begaben wir uns über Neuwied und Zeist nach London und von da nach Bedford.
Am 25. Februar 1826 langten wir nach 15 Wochen auf der stürmischen See in Kapstadt an.
Wir reisten nun über Grönekloof nach Gnadenthal.
Im Februar 1828 reisten wir zuvorderst nach der Kapstadt.
Wir verließen im März 1829 Gnadenthal.
Unser Weg führte uns zuerst nach Enon.
Am 17. August 1830 verließen wir Silo.
Nach einer zwölftägigen Reise in Enon an.
Als wir im November 1839 in Enon anlangten, sah es da selbst gar traurig aus.
Nachdem derselbe in Ebersdorf mit der ledigen Schwester Rosalie Bauer verbunden worden und diese unsere geliebten Kinder sich noch einige Zeit bei uns aufhielten, sah ich dieselben mit dankbar gebeugtem Herzen ihrem hohen Berufe entgegen gehen.
Johannes Lemmerz, heimgegangen in Kleinwelke den 6. Mai 1855."""

In [77]:
doc = ner_model(text)

In [78]:
# Erkenne und gebe die Entitäten aus
for ent in doc.ents:
    print(ent.text, ent.label_)

Jahr 1799 DATE
Jahr 1802 DATE
Heidelberg LOC
Gnadau LOC
März 1807 DATE
Herrnhut LOC
Januar 1809 DATE
April 1815 DATE
Südafrika LOC
Juli wurden DATE
August langten DATE
London LOC
September verließen DATE
December langten DATE
Heidenfest 1816 DATE
Gnadenthal LOC
März desselben DATE
Agnes Jenke PER
März wurden DATE
Februar 1817 DATE
Enon LOC
Januar 1822 DATE
Bischof Gottlob Martin Schneider PER
Anfangs Februar 1825 PER
London LOC
Mai in DATE
Kleinwelke LOC
Februar 1826 DATE
Kapstadt LOC
Februar 1828 DATE
März 1829 DATE
Gnadenthal LOC
August 1830 DATE
Enon LOC
November 1839 DATE
Enon LOC
Ebersdorf LOC
Rosalie Bauer PER
Johannes Lemmerz PER
Kleinwelke LOC
Mai LOC


In [81]:
nlp = spacy.load("de_core_news_lg")


# Funktion zum Laden der Testdaten aus einer JSON-Datei
def load_test_data(json_file_path):
    with open(json_file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
        
    test_data = []
    for item in data:
        text = item[0]  # Der Text ist das erste Element der Liste
        entities = item[1].get("entities", [])  # Entitäten befinden sich im zweiten Element
        test_data.append([text, {"entities": entities}])

    return test_data

# Beispiel zum Laden der Testdaten
json_file_path = 'test_data.json'  # Ersetze dies mit dem tatsächlichen Pfad zu deiner JSON-Datei

# Lade die Testdaten aus der JSON-Datei
TEST_DATA = load_test_data(json_file_path)

# Modell evaluieren
results = evaluate(nlp, TEST_DATA)

# Ergebnisse ausdrucken
for key, val in results.items():
    print(f"{key}: {val}")

token_acc: 1.0
token_p: 1.0
token_r: 1.0
token_f: 1.0
sents_p: None
sents_r: None
sents_f: None
tag_acc: None
pos_acc: None
morph_acc: None
morph_micro_p: None
morph_micro_r: None
morph_micro_f: None
morph_per_feat: None
dep_uas: None
dep_las: None
dep_las_per_type: None
ents_p: 0.14814814814814814
ents_r: 0.5
ents_f: 0.22857142857142856
ents_per_type: {'MISC': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'DATE': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'LOC': {'p': 0.23529411764705882, 'r': 1.0, 'f': 0.38095238095238093}, 'PER': {'p': 0.0, 'r': 0.0, 'f': 0.0}}
cats_score: 0.0
cats_score_desc: macro F
cats_micro_p: 0.0
cats_micro_r: 0.0
cats_micro_f: 0.0
cats_macro_p: 0.0
cats_macro_r: 0.0
cats_macro_f: 0.0
cats_macro_auc: 0.0
cats_f_per_type: {}
cats_auc_per_type: {}


In [ ]:


# Funktion zum Laden der Testdaten aus einer JSON-Datei
def load_test_data(json_file_path):
    with open(json_file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
        
    test_data = []
    for item in data:
        text = item[0]  # Der Text ist das erste Element der Liste
        entities = item[1].get("entities", [])  # Entitäten befinden sich im zweiten Element
        test_data.append([text, {"entities": entities}])

    return test_data

# Beispiel zum Laden der Testdaten
json_file_path = 'test_data.json'  # Ersetze dies mit dem tatsächlichen Pfad zu deiner JSON-Datei

# Lade die Testdaten aus der JSON-Datei
TEST_DATA = load_test_data(json_file_path)

# Modell evaluieren
results = evaluate(ner_model, TEST_DATA)

# Ergebnisse ausdrucken
for key, val in results.items():
    print(f"{key}: {val}")


token_acc: 1.0
token_p: 1.0
token_r: 1.0
token_f: 1.0
sents_p: None
sents_r: None
sents_f: None
tag_acc: None
pos_acc: None
morph_acc: None
morph_micro_p: None
morph_micro_r: None
morph_micro_f: None
morph_per_feat: None
dep_uas: None
dep_las: None
dep_las_per_type: None
ents_p: 0.25
ents_r: 0.375
ents_f: 0.3
ents_per_type: {'DATE': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'LOC': {'p': 0.42857142857142855, 'r': 0.75, 'f': 0.5454545454545454}, 'PER': {'p': 0.0, 'r': 0.0, 'f': 0.0}}
cats_score: 0.0
cats_score_desc: macro F
cats_micro_p: 0.0
cats_micro_r: 0.0
cats_micro_f: 0.0
cats_macro_p: 0.0
cats_macro_r: 0.0
cats_macro_f: 0.0
cats_macro_auc: 0.0
cats_f_per_type: {}
cats_auc_per_type: {}


In [ ]:
from spacy.training.example import Example

# Modell laden
nlp = spacy.load("de_core_news_lg")

# Funktion zum Laden der Testdaten als spaCy Examples
def load_test_data_as_examples(json_file_path, nlp):
    with open(json_file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
    
    examples = []
    for item in data:
        text = item[0]
        annotations = item[1]
        
        doc = nlp.make_doc(text)  # Nur Tokenisierung, keine Vorhersagen
        example = Example.from_dict(doc, annotations)
        examples.append(example)
    
    return examples

# Lade Testdaten
json_file_path = "test_data.json"
examples = load_test_data_as_examples(json_file_path, nlp)

# Modell evaluieren
results = nlp.evaluate(examples)

# Ergebnisse ausgeben
for key, val in results.items():
    print(f"{key}: {val}")


token_acc: 1.0
token_p: 1.0
token_r: 1.0
token_f: 1.0
tag_acc: None
pos_acc: None
morph_acc: None
morph_micro_p: None
morph_micro_r: None
morph_micro_f: None
morph_per_feat: None
sents_p: None
sents_r: None
sents_f: None
dep_uas: None
dep_las: None
dep_las_per_type: None
lemma_acc: None
ents_p: 0.14814814814814814
ents_r: 0.5
ents_f: 0.22857142857142856
ents_per_type: {'MISC': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'DATE': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'LOC': {'p': 0.23529411764705882, 'r': 1.0, 'f': 0.38095238095238093}, 'PER': {'p': 0.0, 'r': 0.0, 'f': 0.0}}
speed: 8762.35563752798


In [86]:
from spacy.training.example import Example
from spacy.tokens import DocBin, Doc
import json

def load_test_data_as_examples(json_file_path, ner_model):
    with open(json_file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)

    examples = []
    for item in data:
        text = item[0]
        annotations = item[1]
        doc = ner_model.make_doc(text)

        ents = []
        for start, end, label in annotations.get("entities", []):
            span = doc.char_span(start, end, label=label, alignment_mode="expand")
            if span is not None:
                ents.append(span)
            else:
                print(f"Warnung: Entität konnte nicht aligned werden: '{text[start:end]}'")

        doc.ents = ents
        example = Example.from_dict(doc, {"entities": [(ent.start_char, ent.end_char, ent.label_) for ent in ents]})
        examples.append(example)

    return examples


In [87]:
# Modell evaluieren
results = ner_model.evaluate(examples)

# Ergebnisse ausgeben
print("\nEvaluationsergebnisse:")
for key, val in results.items():
    if isinstance(val, float):
        print(f"{key}: {val:.4f}")
    else:
        print(f"{key}: {val}")



Evaluationsergebnisse:
token_acc: 1.0000
token_p: 1.0000
token_r: 1.0000
token_f: 1.0000
ents_p: 0.2500
ents_r: 0.3750
ents_f: 0.3000
ents_per_type: {'DATE': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'LOC': {'p': 0.42857142857142855, 'r': 0.75, 'f': 0.5454545454545454}, 'PER': {'p': 0.0, 'r': 0.0, 'f': 0.0}}
speed: 15638.0315


In [31]:
import json
import spacy
from spacy.tokens import DocBin

with open("output/training_data.json", "r", encoding="utf-8") as f:
    training_data = json.load(f)

nlp = spacy.blank("de")
doc_bin = DocBin()

for text, annot in training_data:
    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print(f"⚠️ Ungültiger Span: ({start}, {end}) → '{text[start:end]}'")
            print(f"→ Ganzer Text: {text}\n")
            continue
        ents.append(span)
    doc.ents = ents
    doc_bin.add(doc)

doc_bin.to_disk("training_data.spacy")
print("✅ Umwandlung abgeschlossen: training_data.spacy gespeichert")


⚠️ Ungültiger Span: (57, 63) → 'Menger'
→ Ganzer Text: Am 15. Juni 1859 eröffnete Johann Philipp George auf dem Menger-Gelände sein neues Unternehmen und durchfahre die ersten Schritte in seinem neuen Lebensabschnitt.

⚠️ Ungültiger Span: (54, 64) → 'Gnadenfrei'
→ Ganzer Text: Anna Charlotte Luther wackelt am 24. Mai 1841 auf dem Gnadenfreier Friedhof, während sie entdeckt, dass die Schönheit des Ortes ihre Seele berührt.

⚠️ Ungültiger Span: (89, 97) → 'Herrnhut'
→ Ganzer Text: Georg Traneker bemüht sich dabei sein, sich anpassen zu können, um als neue Seele in der Herrnhuter Gemeinschaft Fuß fassen zu können, bevor er im September 1867 nach Herrnhut kommen möchte und seine Heimat verlassen will.

⚠️ Ungültiger Span: (84, 89) → 'Acker'
→ Ganzer Text: Am 15. Dezember 1883 begreift Johanne Gysbertine Eliſabeth Pronkert die Chancen des Ackerbodens für ihre neue landwirtschaftliche Initiative und vorantreibt ihre Pläne, um den Ertrag zu steigern.

⚠️ Ungültiger Span: (117, 130) → 'Neudiet

ValueError: [E1010] Unable to set entity information for token 2 which is included in more than one span in entities, blocked, missing or outside.